# Set up for Parking Sensors

TO DO: 
- Need to update the landing page references
- Add unit test cases in a separate notebook

About:

- This notebook creates the database schemas and tables needed by parking sensors sample. See [parking sensors page](https://github.com/Azure-Samples/modern-data-warehouse-dataops/tree/feat/e2e-fabric-dataops-sample/e2e_samples/fabric_dataops_sample/README.md) for more details about Parking Sensor sample using Microsoft Fabric.

Assumptions/Pre-requisites:

- User must set appropriate flags in the environment in case "case-sensitive" names need to be maintained for tables and columns. 
    ```
    spark.sql.caseSensitive: 'TRUE'
    ```
- Currently there is a known issue running cross workspace queries when workspace name has special characters. See [schema limitation](https://learn.microsoft.com/en-us/fabric/data-engineering/lakehouse-schemas#public-preview-limitations) for more details. Avoid special characters if planning to query across workspaces with schema support. 
- The following assets are created by IaC step:
    - The required files are staged under "Files/" of the lakehouse (to be added as a default lakehouse during runtime) as follows:
        - Files/sc-adls-main/config/lakehouse_ddls.yaml 
        - Files/sc-adls-main/config/application.cfg (derived using application.cfg.template during ci/cd process). Ensure "setup" section is updated with the required parameters for this notebook.
        - Files/sc-adls-main/reference/dim_date.csv
        - Files/sc-adls-main/reference/dim_time.csv 
    - Environment with common library otel_monitor_invoker.py and its associated python dependencies
    - Parking Sensor Lakehouse
    - Datasource: ADLS made available as a shortcut in Parking Sensor Lakehouse
    - Monitoring sink: AppInsights
    - Secrets repo: Key vault to store AppInsights connection information

- All Lakehouses have schema support enabled (in Public preview as of Nov, 2024).
- Execution
  - A default lakehouse is associated during runtime where the required files and data are already staged. Multiple ways of invoking:
    - [Api call](https://learn.microsoft.com/fabric/data-engineering/notebook-public-api#run-a-notebook-on-demand)
    - [Part of a data pipeline](https://learn.microsoft.com/fabric/data-engineering/author-execute-notebook#parameterized-session-configuration-from-a-pipeline)
    - [Using `%run` from another notebook](https://learn.microsoft.com/fabric/data-engineering/author-execute-notebook#reference-run-a-notebook)


## Parameters and Library imports

### Reading parameters (external from Fabric pipeline or default values)

In [ ]:
%%configure
{
    "defaultLakehouse": {
        "name": {
            "parameterName": "lakehouse_name",
            "defaultValue": "{{ .lakehouse_name }}"
               } ,
        "id": {
            "parameterName": "lakehouse_id",
            "defaultValue": "{{ .lakehouse_id }}"
        } ,
        "workspaceId": {
            "parameterName": "workspace_id",
            "defaultValue": "{{ .workspace_id }}"
        }
    },
    "mountPoints": [
        {
            "mountPoint": "/local_data",
            "source": {
                "parameterName": "local_mount",
                "defaultValue": "abfss://{{ .workspace_id }}@onelake.dfs.fabric.microsoft.com/{{ .lakehouse_id }}/Files"
            }
        }
    ]
}

In [ ]:
# Unless `%%configure` is used to read external parameters - this cell should be the first one

# This cell is tagged as Parameters cell. Parameters mentioned here are usually \
#    passed by the user at the time of notebook execution.
# Ref: https://learn.microsoft.com/fabric/data-engineering/notebook-public-api#run-a-notebook-on-demand

# Control how to run the notebook - "all" for entire notebook or "module" mode to use \
#      this notebook like a module (main execution will be skipped). Useful when performing
#      testing using notebooks or functions from this notebook need to be called from another notebook.
import configparser
import json
from datetime import datetime

import otel_monitor_invoker as otel  # custom module part of env
import yaml  # type: ignore
from opentelemetry.trace import SpanKind
from opentelemetry.trace.status import StatusCode

# execution_mode = "module" will skip the execution of the main function. Use it for module like treatment
#   "all" perform execution as well.
execution_mode = "all"
# Helpful if user wants to set a child process name etc. will be derived if not set by user
job_exec_instance = ""
# Helpful to derive any stage based globals
env_stage = "dev"
# Common config file path hosted on attached lakehouse - path relative to Files/
config_file_path = "sc-adls-main/config/application.cfg"

In [ ]:
# Validate input parameters
in_errors = []
if execution_mode not in ["all", "module"]:
    in_errors.append(f"Invalid value: {execution_mode = }. It must be either 'all' or 'module'.")
if not notebookutils.fs.exists(f"Files/{config_file_path}"):
    in_errors.append(f"Specified config - `Files/{config_file_path}` doesn't exist.")

if in_errors:
    raise ValueError(f"Input parameter validation failed. Errors are:\n{in_errors}")
else:
    print("Input parameter verification completed successfully.")

### Network mounts

- Scope is set to Job/session - so these need to be run once per session


In [ ]:
# -- Helps to read config files from onelake location
runtime_context = notebookutils.runtime.context
# Mount is configured already using %%configure.
# notebookutils.fs.mount(
#     source=f"abfss://{runtime_context['currentWorkspaceId']}@onelake.dfs.fabric.microsoft.com/{runtime_context['defaultLakehouseId']}/Files",
#     mountPoint="/local_data",
#     extraConfigs={"Scope": "job"},
# )
local_data_mount_path = f'{notebookutils.fs.getMountPath("/local_data")}'

### Read user provided config values


In [ ]:
config = configparser.ConfigParser(interpolation=configparser.ExtendedInterpolation())
config.read(f"{local_data_mount_path}/{config_file_path}")

In [ ]:
# When we config parser if the value is not present in the specified section, it will be
#   read from "DEFAULT" section.
config_section_name = "setup"
process_name = config.get(config_section_name, "process_name")
parking_ws = config.get(config_section_name, "workspace_name")
parking_ws_id = config.get(config_section_name, "workspace_id")
parking_lakehouse = config.get(config_section_name, "lakehouse_name")
ddl_config_file = config.get(config_section_name, "ddl_file")  # relative path to Files/

### Internal (derived) parameters

In [ ]:
# default is micro-seconds, changing to milliseconds
current_ts = datetime.utcnow().strftime("%Y%m%d%H%M%S%f")[:-3]
job_exec_instance = job_exec_instance if job_exec_instance else f"{process_name}#{current_ts}"
execution_user_name = runtime_context["userName"]

## Monitoring and observability

### AppInsights connection

In [ ]:
connection_string = notebookutils.credentials.getSecret(
    config.get("keyvault", "uri"), config.get("otel", "appinsights_connection_name")
)
otlp_exporter = otel.OpenTelemetryAppInsightsExporter(conn_string=connection_string)

### Populate resource information

In [ ]:
# Resource references
# - Naming conventions: https://opentelemetry.io/docs/specs/semconv/general/attribute-naming/
# - For a complete list of reserved ones: https://opentelemetry.io/docs/concepts/semantic-conventions/
#  NOTE: service.namespace,service.name,service.instance.id triplet MUST be globally unique.
#     The ID helps to distinguish instances of the same service that exist at the same time
#     (e.g. instances of a horizontally scaled service)
resource_attributes = {
    # ---------- Reserved attribute names
    "service.name": config.get(config_section_name, "service_name"),
    "service.version": config.get(config_section_name, "service_version"),
    "service.namespace": "parking-sensor",
    "service.instance.id": notebookutils.runtime.context["activityId"],
    "process.executable.name": process_name,
    "deployment.environment": env_stage,
    # ---------- custom attributes - we can also add common attributes like appid, domain id etc
    #     here or get them from process reference data using process name as the key.
    # runtime context has a lot if useful info - adding it as is.
    "jobexec.context": f"{notebookutils.runtime.context}",  # convert to string otherwise it will fail
    "jobexec.cluster.region": spark.sparkContext.getConf().get("spark.cluster.region"),
    "jobexec.app.name": spark.sparkContext.getConf().get("spark.app.name"),
    "jobexec.instance.name": job_exec_instance,
}

# Typically, logging is performed within the context of a span.
#   This allows log messages to be associated with trace information through the use of trace IDs and span IDs.
#   As a result, it's generally not necessary to include resource information in log messages.
# Note that trace IDs and span IDs will be null when logging is performed outside of a span context.
log_attributes = {"jobexec.instance.name": job_exec_instance}
trace_attributes = resource_attributes

tracer = otlp_exporter.get_otel_tracer(trace_resource_attributes=trace_attributes, tracer_name=f"tracer-{process_name}")
logger = otlp_exporter.get_otel_logger(
    log_resource_attributes=log_attributes,
    logger_name=f"logger-{process_name}",
    add_console_handler=False,
)
logger.setLevel("INFO")  # default is WARN

## Code

### Code functions

- When using %run we can expose these functions to the calling notebook.

In [ ]:
def get_lakehouse_details(lakehouse_name: str) -> dict:
    logger.info("Performing lakehouse existence check.")
    try:
        details = notebookutils.lakehouse.get(name=lakehouse_name)
    except Exception:
        logger.exception(f"Specified lakehouse - {lakehouse_name} doesn't exist. Aborting..")
        raise
    return details


def create_schemas(schema_list: list) -> None:
    errors = []
    try:
        for schema in schema_list:
            logger.info(f"Recreating schema: {parking_lakehouse}.{schema}")
            spark.sql(f"create schema if not exists {parking_lakehouse}.{schema}")
    except Exception:
        error_msg = f"Failed to create schema{parking_lakehouse}.{schema}"
        errors.append(error_msg)
        logger.exception(error_msg)
    if errors:
        raise ValueError(f"There are errors:\n{errors}")
    else:
        return None


def create_tables(ddl_dict: dict, lakehouse_table_path: str, lakehouse_file_path: str) -> None:
    # Update in future - special characters in workspace are causing issues
    #  use f"{parking_ws}.{parking_lakehouse}"  if possible
    lake_house_prefix = parking_lakehouse
    errors = []
    for t in ddl_dict["table_types"]:
        logger.info(f"\nProcessing type: {t['type']}")
        for table in t["tables"]:
            table_name = table["table_name"].strip()
            table_fqn = f"{lake_house_prefix}.{table_name}"
            logger.info(f"\t- Processing table:{table_fqn}")
            data_file = table.get("data_file", None)
            create_sql = table.get("create_sql", None)
            drop_sql = f"DROP TABLE IF EXISTS {table_fqn}"
            try:
                if data_file:
                    data_file = f"{lakehouse_file_path}/{data_file}".strip()
                    spark.read.csv(data_file, header=True).write.format("delta").mode("overwrite").save(
                        f"{lakehouse_table_path}/{table_name.replace('.', '/')}"
                    )
                elif create_sql:
                    create_sql = create_sql.format(workspace_name=parking_ws, lakehouse_name=parking_lakehouse)
                    spark.sql(drop_sql)
                    spark.sql(create_sql)
                else:
                    raise ValueError("Either `data_file` or `create_sql` must be defined.")

            except Exception as e:
                logger.exception(f"\t\t- Failed processing table:{table_fqn}")
                errors.append(
                    {
                        "table_name": table_fqn,
                        "data_file": data_file,
                        "drop_sql": drop_sql,
                        "create_sql": create_sql,
                        "error": e,
                    }
                )
    if errors:
        raise ValueError(
            f"Encountered errors while creating tables. Errors are\n{json.dumps(errors, indent=2, default=str)}"
        )
    return None

In [ ]:
def main() -> None:
    root_span_name = f"root#{process_name}#{current_ts}"

    with tracer.start_as_current_span(root_span_name, kind=SpanKind.INTERNAL) as root_span:
        try:
            root_span.add_event(
                name="010-verify-lakehouse",
                attributes={"lakehouse_name": parking_lakehouse},
            )
            lh_details = get_lakehouse_details(parking_lakehouse)
            lh_table_path = f'{lh_details["properties"]["abfsPath"]}/Tables'
            lh_file_path = f'{lh_details["properties"]["abfsPath"]}/Files'

            ddl_file_path = f"{local_data_mount_path}/{ddl_config_file}"
            root_span.add_event(name="020-read-ddls", attributes={"ddl_file": ddl_file_path})
            with open(ddl_file_path, "r") as d:
                ddl_dict = yaml.safe_load(d.read())

            root_span.add_event(name="030-schema-creation", attributes={})
            create_schemas(["dw", "lnd", "interim", "malformed"])

            root_span.add_event(name="040-table-creation", attributes={})
            create_tables(ddl_dict, lh_table_path, lh_file_path)

        except Exception as e:
            error_message = f"{process_name} process failed with error {e}"
            logger.exception(error_message)
            root_span.set_status(StatusCode.ERROR, error_message)
            root_span.record_exception(e)
            raise
        else:
            root_span.set_status(StatusCode.OK)
            logger.info(f"{process_name} process is successful.")
        finally:
            logger.info(f"\n** {process_name} process is complete. Check the logs for execution status. **\n\n")

    return None

### Code execution

In [ ]:
# Apply logic here incase this notebook need to be used as a library

# dependencies
# | where name hasprefix "root#nb-020"
# //
# exceptions
# //
# traces


if execution_mode == "all":
    print(f"{execution_mode = }. Proceeding with the code execution.")
    main()
else:
    print(f"Skipping the main function execution as {execution_mode = } and running it like a code module.")